In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression



from sklearn.metrics import mutual_info_score

%matplotlib inline

In [2]:
df = pd.read_csv("./data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


### Data Preparation

In [4]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

categorical_columns = list(df.dtypes[df.dtypes == "object"].index)
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(" ", "_")

In [5]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [6]:
tc = pd.to_numeric(df.totalcharges, errors="coerce")

In [7]:
df[tc.isnull()][["customerid", "totalcharges"]]

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [8]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors="coerce").fillna(0)

In [9]:
df[tc.isnull()][["customerid", "totalcharges"]]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


In [10]:
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [11]:
df.churn = (df.churn == "yes").astype(int)

In [12]:
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int32

### Setting up validation framework

In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=142)

len(df_full_train), len(df_test)

(5634, 1409)

In [14]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=142)

len(df_train), len(df_val)

(4225, 1409)

In [15]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [17]:
del df_train["churn"]
del df_val["churn"]
del df_test["churn"]

### EDA

In [18]:
df_full_train = df_full_train.reset_index(drop=True)

In [19]:
df_full_train.isna().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [20]:
df.churn.value_counts(normalize=True)

0    0.73463
1    0.26537
Name: churn, dtype: float64

In [21]:
global_churn_rate = round(df_full_train.churn.mean(), 2)
global_churn_rate

0.26

In [22]:
numerical = ["tenure", "monthlycharges", "totalcharges"]
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice',  'multiplelines', 
                'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 
                'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']

In [23]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### Feature Importance: Chgurn Rate & Risk Ratio

In [24]:
df_full_train[df_full_train.gender == "female"].churn.mean()/global_churn_rate

1.0096673222348411

In [25]:
df_full_train[df_full_train.gender == "male"].churn.mean()/global_churn_rate

1.023128079480897

In [26]:
df_full_train[df_full_train.partner == "yes"].churn.mean()/global_churn_rate

0.7440054418683748

In [27]:
df_full_train[df_full_train.partner == "no"].churn.mean()/global_churn_rate

1.2697576396206531

In [28]:
df_full_train[df_full_train.dependents == "yes"].churn.mean()/global_churn_rate

0.576575742138656

In [29]:
df_full_train[df_full_train.dependents == "no"].churn.mean()/global_churn_rate

1.2004104629324865

In [30]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(["mean", "count"])
    df_group["mean"] = round(df_group["mean"], 2)
    df_group["diff"] = round(df_group["mean"] - global_churn_rate, 2)
    df_group["risk"] = round(df_group["mean"] / global_churn_rate, 2)
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.26,2777,0.00,1.00
male,0.27,2857,0.01,1.04




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.23,4726,-0.03,0.88
1,0.42,908,0.16,1.62




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.43,3120,0.17,1.65
one_year,0.11,1184,-0.15,0.42
two_year,0.02,1330,-0.24,0.08




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.16,2297,-0.10,0.62
yes,0.33,3337,0.07,1.27




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.16,1249,-0.10,0.62
credit_card_(automatic),0.15,1211,-0.11,0.58
electronic_check,0.45,1875,0.19,1.73
mailed_check,0.20,1299,-0.06,0.77


### Feature Importance: Mutual Information

In [31]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.10069113679294704

In [32]:
mutual_info_score(df_full_train.churn, df_full_train.gender)

7.873040017092325e-06

In [33]:
mutual_info_score(df_full_train.churn, df_full_train.totalcharges)

c:\Users\soumy\anaconda3\envs\mlops\lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


0.5425289427149174

In [34]:
mutual_info_score(df_full_train.churn, df_full_train.partner)

0.012153344490082568

In [35]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

contract            0.100691
onlinesecurity      0.068576
techsupport         0.062718
internetservice     0.054185
onlinebackup        0.044742
paymentmethod       0.044284
deviceprotection    0.043403
streamingmovies     0.030532
streamingtv         0.030474
paperlessbilling    0.019181
dependents          0.015158
partner             0.012153
seniorcitizen       0.010899
multiplelines       0.000775
phoneservice        0.000034
gender              0.000008
dtype: float64

### Feature Importance: Correlation

In [36]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.356726
monthlycharges    0.188541
totalcharges     -0.203819
dtype: float64

In [37]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5830945558739254

In [38]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure < 12)].churn.mean()

0.41133263378803775

In [39]:
df_full_train[df_full_train.tenure >= 12].churn.mean()

0.1732362540798393

In [40]:
df_full_train[numerical].corrwith(df_full_train.churn).abs()

tenure            0.356726
monthlycharges    0.188541
totalcharges      0.203819
dtype: float64

### One Hot Encoding

In [41]:
train_dict  = df_train[categorical + numerical].to_dict(orient="records")
train_dict[0]

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'no',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'no',
 'onlinesecurity': 'no_internet_service',
 'onlinebackup': 'no_internet_service',
 'deviceprotection': 'no_internet_service',
 'techsupport': 'no_internet_service',
 'streamingtv': 'no_internet_service',
 'streamingmovies': 'no_internet_service',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'credit_card_(automatic)',
 'tenure': 61,
 'monthlycharges': 25.0,
 'totalcharges': 1498.35}

In [42]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [43]:
X_train = dv.transform(train_dict)

In [44]:
X_train.shape

(4225, 45)

In [45]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

In [46]:
X_train[1]

array([0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 2.4450e+01, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 5.9000e+01, 1.4931e+03])

In [47]:
val_dict = df_val[categorical + numerical].to_dict(orient="records")
test_dict = df_test[categorical + numerical].to_dict(orient="records")

X_val = dv.transform(val_dict)
X_test = dv.transform(test_dict)

### Logistic Regression

In [49]:
model = LogisticRegression()

model.fit(X_train, y_train)

LogisticRegression()

In [52]:
model.coef_[0].round(3)

array([ 0.464, -0.166, -0.419, -0.017, -0.103,  0.036, -0.094, -0.063,
       -0.082, -0.039, -0.362,  0.335, -0.094,  0.001, -0.24 ,  0.123,
       -0.004,  0.05 , -0.094, -0.077,  0.321, -0.094, -0.348, -0.253,
        0.132, -0.085, -0.035, -0.139, -0.125,  0.252, -0.11 ,  0.123,
       -0.244,  0.256, -0.083, -0.094,  0.057, -0.122, -0.094,  0.095,
        0.258, -0.094, -0.285, -0.063,  0.   ])

In [53]:
model.intercept_.round(3)

array([-0.122])

In [58]:
model.predict_proba(X_train)[:,1]

array([0.00651276, 0.00795327, 0.33082826, ..., 0.20787705, 0.0384342 ,
       0.05226429])

In [60]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.0824369 , 0.05101282, 0.60881115, ..., 0.00445324, 0.00186203,
       0.46680639])

In [64]:
churn_decision = (y_pred >= 0.5)
churn_decision

array([False, False,  True, ..., False, False, False])

In [65]:
(churn_decision.astype(int) ==y_val).mean()

0.8183108587650816

In [66]:
df_val[churn_decision].customerid

2       3398-zouaa
5       8120-jdcam
7       8242-soquo
10      3727-rjmeo
16      9689-ptnpg
           ...    
1377    3177-lasxd
1381    2495-inzwq
1384    3096-gkweb
1388    7924-gjzfi
1397    5376-deqcp
Name: customerid, Length: 311, dtype: object

In [67]:
df_pred = pd.DataFrame()
df_pred["probability"] = y_pred
df_pred["prediction"] = churn_decision.astype(int)
df_pred["actual"] = y_val

In [68]:
df_pred["correct"] = (df_pred["prediction"] == df_pred["actual"])

In [69]:
df_pred["correct"].astype(int).mean()

0.8183108587650816

### Model Interpretation